In [ ]:
# install ffmpeg
# !conda install -y -q ffmpeg

In [ ]:
# optional: confirm that ffmpeg is installed
# ! ffmpeg

In [ ]:
# !pip install --user jiwer
# !pip install --user openai-whisper

In [ ]:
%pip install git+https://github.com/m-bain/whisperx.git

In [ ]:
HUGGINGFACE_TOKEN = ""

In [ ]:
import whisperx
import gc 
from whisperx import load_align_model, align
from whisperx.diarize import DiarizationPipeline, assign_word_speakers

In [2]:
import os
mp3_file = "./2016_07_15.mp3"
print(f"mp3_file exists: {os.path.exists(mp3_file)}")

mp3_file exists: True


In [ ]:
%%time
device = "cuda" 
audio_file = mp3_file
batch_size = 1 # reduce if low on GPU mem
compute_type = "float16" # change to "int8" if low on GPU mem (may reduce accuracy)

# 1. Transcribe with original whisper (batched)
# model = whisperx.load_model("medium", device, compute_type=compute_type)
model = whisper.load_model("medium")

audio = whisperx.load_audio(audio_file)
# result = model.transcribe(audio, batch_size=batch_size)
result = model.transcribe(audio_file, fp16=False)
print(result["segments"]) # before alignment
# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model



In [ ]:
%%time
# 2. Align whisper output
model_a, metadata = whisperx.load_align_model(language_code=result["language"], device=device)
result = whisperx.align(result["segments"], model_a, metadata, audio, device, return_char_alignments=False)

print(result["segments"]) # after alignment

# delete model if low on GPU resources
# import gc; gc.collect(); torch.cuda.empty_cache(); del model_a

In [ ]:
%%time
# 3. Assign speaker labels
diarize_model = whisperx.DiarizationPipeline(use_auth_token=os.environ["HUGGINGFACE_TOKEN"], device=device)

# add min/max number of speakers if known
# diarize_segments = diarize_model(audio)
diarize_segments = diarize_model(audio, min_speakers=2)
# diarize_model(audio, min_speakers=min_speakers, max_speakers=max_speakers)

result2 = whisperx.assign_word_speakers(diarize_segments, result)
# print(diarize_segments)

print(f'Speakers: {list(set([item["speaker"] for item in result2["segments"]]))}')
print(result2["segments"]) # segments are now assigned speaker IDs